### Load tensorflow

In [1]:
import tensorflow as tf

/usr/local/anaconda/python3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data (prices.csv)

In [3]:
import pandas as pd

data = pd.read_csv('prices.csv')

In [4]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [5]:
data.isna().sum()

date      0
symbol    0
open      0
close     0
low       0
high      0
volume    0
dtype: int64

### Check all columns in the dataset

In [6]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Drop columns `date` and  `symbol`

In [7]:
data = data.drop(['date', 'symbol'], axis=1)

In [8]:
data.shape

(851264, 5)

### Consider only first 1000 rows in the dataset for building feature set and target set

In [9]:
subdata = data.head(1000)
print(subdata.shape)

(1000, 5)


### Divide the data into train and test sets

In [10]:
from sklearn.model_selection import train_test_split

features = subdata.drop('volume', axis=1)
target = subdata['volume']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

In [11]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(700, 4) (300, 4) (700,) (300,)


## Building the graph in tensorflow

1.Define input data placeholders

In [12]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.layers.batch_normalization(x, training=True)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [13]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

3.Prediction

In [14]:
y = tf.add(tf.matmul(x_n, W), b, name='output')

4.Loss (Cost) Function [Mean square error]

In [15]:
loss = tf.reduce_mean(tf.square(y-y_), name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [16]:
train_op = tf.train.GradientDescentOptimizer(0.0000001).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [17]:
sess = tf.Session()

In [18]:
sess.run(tf.global_variables_initializer())

In [20]:
training_epochs = 100

### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [21]:
sess.run(W)

array([[0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [22]:
sess.run(b)

array([0.], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [23]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op, loss], feed_dict={x:X_train, y_:y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)
        print (sess.run(W))

Training loss at step:  0  is  250625820000000.0
[[ 2.7187500e-07]
 [-2.3046876e-07]
 [-9.0898436e-07]
 [ 5.5429689e-07]]
Training loss at step:  10  is  250625800000000.0
[[ 0.00034011]
 [-0.00028797]
 [-0.00116109]
 [ 0.00071382]]
Training loss at step:  20  is  250625590000000.0
[[ 0.44493145]
 [-0.37676078]
 [-1.5189812 ]
 [ 0.93388295]]
Training loss at step:  30  is  238562970000000.0
[[  485.7965 ]
 [ -411.36224]
 [-1658.4819 ]
 [ 1019.6501 ]]
Training loss at step:  40  is  224073430000000.0
[[  475.79114]
 [ -402.88983]
 [-1624.3239 ]
 [  998.6495 ]]
Training loss at step:  50  is  224101610000000.0
[[  474.65793]
 [ -401.93   ]
 [-1620.4557 ]
 [  996.2711 ]]
Training loss at step:  60  is  224103910000000.0
[[  474.56747]
 [ -401.85297]
 [-1620.1461 ]
 [  996.08044]]
Training loss at step:  70  is  224104100000000.0
[[  474.5606 ]
 [ -401.84732]
 [-1620.1244 ]
 [  996.0661 ]]
Training loss at step:  80  is  224104100000000.0
[[  474.55963]
 [ -401.84628]
 [-1620.1195 ]
 [  99

In [24]:
sess.close()

In [25]:
tf.reset_default_graph()

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [26]:
model = tf.keras.models.Sequential()

#model.add(tf.keras.layers.InputLayer(input_shape=(4,)))
#model.add(tf.keras.layers.Reshape((4,),input_shape=(4,)))
#model.add(tf.keras.layers.Dense())
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='mse', metrics=['accuracy'])

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 4)                 16        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 21
Trainable params: 13
Non-trainable params: 8
_________________________________________________________________


### Execute the model

In [28]:
model.fit(X_train, y_train, epochs=30)

Epoch 1/30
700/700 [==============================] - 0s 297us/step - loss: 250649149117937.3750 - acc: 0.0000e+00
Epoch 2/30
700/700 [==============================] - 0s 37us/step - loss: 250649132927923.9062 - acc: 0.0000e+00
Epoch 3/30
700/700 [==============================] - 0s 38us/step - loss: 250649143785179.4375 - acc: 0.0000e+00
Epoch 4/30
700/700 [==============================] - 0s 38us/step - loss: 250649139015656.6250 - acc: 0.0000e+00
Epoch 5/30
700/700 [==============================] - 0s 36us/step - loss: 250649145295128.8750 - acc: 0.0000e+00
Epoch 6/30
700/700 [==============================] - 0s 36us/step - loss: 250649141340499.3750 - acc: 0.0000e+00
Epoch 7/30
700/700 [==============================] - 0s 39us/step - loss: 250649155145751.3750 - acc: 0.0000e+00
Epoch 8/30
700/700 [==============================] - 0s 37us/step - loss: 250649137565625.8125 - acc: 0.0000e+00
Epoch 9/30
700/700 [==============================] - 0s 37us/step - loss: 250649141484

### Build a 2 layer neural network with input layer, 5 neurons in first hidden layer and one neuron in the output layer.

In [29]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))
model.add(tf.keras.layers.Dense(50, activation='sigmoid'))
model.add(tf.keras.layers.Dense(100, activation='sigmoid'))
model.add(tf.keras.layers.Dense(50, activation='sigmoid'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

sgd_optimizer = tf.keras.optimizers.SGD(lr=0.00003)
model.compile(optimizer=sgd_optimizer, loss='mse', metrics=['accuracy'])


In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 4)                 16        
_________________________________________________________________
dense_2 (Dense)              (None, 50)                250       
_________________________________________________________________
dense_3 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 10,467
Trainable params: 10,459
Non-trainable params: 8
_________________________________________________________________


In [32]:
model.fit(X_train, y_train, epochs=30)

Epoch 1/30
700/700 [==============================] - 0s 50us/step - loss: 250649136618911.4688 - acc: 0.0000e+00
Epoch 2/30
700/700 [==============================] - 0s 46us/step - loss: 250649147763776.3438 - acc: 0.0000e+00
Epoch 3/30
700/700 [==============================] - 0s 46us/step - loss: 250649137691454.9062 - acc: 0.0000e+00
Epoch 4/30
700/700 [==============================] - 0s 46us/step - loss: 250649129260903.8750 - acc: 0.0000e+00
Epoch 5/30
700/700 [==============================] - 0s 46us/step - loss: 250649137433804.7812 - acc: 0.0000e+00
Epoch 6/30
700/700 [==============================] - 0s 46us/step - loss: 250649139758647.5938 - acc: 0.0000e+00
Epoch 7/30
700/700 [==============================] - 0s 46us/step - loss: 250649132736184.3125 - acc: 0.0000e+00
Epoch 8/30
700/700 [==============================] - 0s 47us/step - loss: 250649142634741.7500 - acc: 0.0000e+00
Epoch 9/30
700/700 [==============================] - 0s 47us/step - loss: 2506491394231

#### Compare the accuracy scores across all the 3 models.